## Import lib

In [1]:
from DataSet import DataSet
from SimGPBO_old import SimGPBO_old
from SimGPBO import SimGPBO
from PostProcessor import PostProcessor
import GPy
import numpy as np 
import matplotlib.pyplot as plt
import copy 

In [2]:
def plot_map(ds: DataSet, vector: np.array, title: str=None) -> None:
    map = np.full((
                        np.max(ds.set['ch2xy'][:,0]),
                        np.max(ds.set['ch2xy'][:,1])
                        ), 
                        np.nan)
    for i in range(ds.set['ch2xy'].shape[0]):
        map[int(ds.set['ch2xy'][i,0]-1),int(ds.set['ch2xy'][i,1]-1)] = vector[i]
    plt.figure()
    plt.imshow(map, cmap='viridis', vmin=np.nanmin(map), vmax=np.nanmax(map))
    plt.colorbar(label="values")
    plt.title(title)
    plt.show()


## Import data

In [3]:
ds = DataSet('../data/','nhp','Cebus1_M1_190221.mat','cebus1')
ds.load_matlab_data() # load data from the dataset_file

## Simulation 

In [4]:
sim = SimGPBO(name = 'tttest_hyp_valid', 
                    ds = copy.deepcopy(ds),
                    AF = 'UCB',
                    NB_REP = 1,
                    NB_IT = 40,
                    KAPPA = 6,
                    NB_RND = 1
                    )

sim.select_emgs([0])

sim.run_simulations(gp_origin='gpytorch', response_type='valid', 
                         hyperparams_storage=True)

Global Progress: 100%|██████████| 1/1 [00:22<00:00, 22.07s/iter]

Elapsed time: 22.08238935470581 seconds
final save of tttest_hyp_valid in:   results/gpbo_gpytorch_UCB_cebus1_tttest_hyp_valid.npz


In [5]:
mean = sim.gp.mean.detach().numpy()
std = sim.gp.std.detach().numpy()
best_f = np.max(sim.gp.mean[sim.best_pred_x_measured[0, 0, 0, 39].item()].numpy())

In [9]:
map_emg = np.full((ds.set['sorted_respMean'].shape[1],
                    np.max(ds.set['ch2xy'][:,0]),
                    np.max(ds.set['ch2xy'][:,1])
                    ), 
                    np.nan)
for i in range(ds.set['ch2xy'].shape[0]):
    map_emg[:,int(ds.set['ch2xy'][i,0]-1),int(ds.set['ch2xy'][i,1]-1)] = ds.set['sorted_respMean'][i,:]
# print(map_emg)

map_mean = np.full((
                    np.max(ds.set['ch2xy'][:,0]),
                    np.max(ds.set['ch2xy'][:,0])
                    ), 
                    np.nan)
for i in range(ds.set['ch2xy'].shape[0]):
    map_mean[int(ds.set['ch2xy'][i,0]-1),int(ds.set['ch2xy'][i,1]-1)] = mean[i]
# print(map_mean)

plt.figure(figsize=(6, 6))
plt.imshow(map_emg[3], cmap='viridis', vmin=np.nanmin(map_emg[3]), vmax=np.nanmax(map_emg[3]))
plt.colorbar(label="values")
plt.title("title")
plt.show()

plt.figure(figsize=(6, 6))
plt.imshow(map_mean, cmap='viridis', vmin=np.nanmin(map_mean), vmax=np.nanmax(map_mean))
plt.colorbar(label="values")
plt.title("title")
plt.show()

In [31]:
print(sim.P_test_x_idx)
print(sim.P_test_y)

tensor([[[[61,  7, 88, 78, 89, 79, 88, 94, 87, 18,  1, 30, 47,  8, 77, 48, 37,
           38, 14, 24, 28, 32, 43, 21, 22, 31, 11, 20, 42, 33, 34, 45, 64, 35,
           83,  3, 40, 17, 56, 59]]]])
tensor([[[[0.0024, 0.0022, 0.0029, 0.0022, 0.0024, 0.0021, 0.0025, 0.0064,
           0.0033, 0.0091, 0.0034, 0.0179, 0.0022, 0.0145, 0.0024, 0.0040,
           0.0028, 0.0043, 0.0060, 0.0099, 0.0102, 0.0324, 0.0132, 0.0372,
           0.0199, 0.0199, 0.0186, 0.0114, 0.0094, 0.0383, 0.0347, 0.0142,
           0.0021, 0.0167, 0.0022, 0.0044, 0.0130, 0.0048, 0.0031, 0.0073]]]],
       dtype=torch.float64)
